In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
# Path to file (for Colab)
PATH = ""

# Dataset
<a href="https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/data">Amazon Dataset</a>

# Data Understanding

In [3]:
# See the shape of all images:
# Main Directory
main_path = PATH + 'AmazonDataset/'


def get_size(folder):
    sizes = []
    # Get from main directory all sub-directories
    for folder_path in os.listdir(folder):
        # Skip
        if (folder_path == '.DS_Store'):
            continue
        # See Train and Test sub-directories
        for folder_name in os.listdir(os.path.join(folder, folder_path)):
            # Skip
            if (folder_name in ['.DS_Store', 'test_v2_file_mapping.csv', 'train_v2.csv']):
                continue
            print(folder_name)
            for filename in os.listdir(os.path.join(folder, folder_path, folder_name)):
                
                # take image
                img = Image.open(os.path.join(folder, folder_path, folder_name, filename))
                # Get image with 
                #print(img.size)
                # Stores data like: (width, height)
                sizes.append(img.size)
            
    print(f'the max width is: {max(sizes[0])}, and the min width is: {min(sizes[0])}')
    print(f'the max height is: {max(sizes[1])}, and the min height is: {min(sizes[1])}')
    print(f'the mean width is: {np.mean(sizes[0])}, and the mean height is: {np.mean(sizes[1])}')

# Call the function
get_size(main_path)

test-jpg
train-jpg
the max width is: 256, and the min width is: 256
the max height is: 256, and the min height is: 256
the mean width is: 256.0, and the mean height is: 256.0


In [4]:
train_path = PATH + 'AmazonDataset/Train/'
test_path = PATH + 'AmazonDataset/Test/'

## See Images

In [5]:
images_folder_train = train_path + '/train-jpg/'
images_folder_test = test_path + '/test-jpg/'

In [6]:
import matplotlib.pyplot as plt
def plot_images(axis=(2,2), images_folder_path=images_folder_train, train = True):
    
    if train:
        train = 'train_'
    else:
        train = 'test_'
        
    # Grid
    f, axarr = plt.subplots(axis[0], axis[1], figsize=(30/axis[1], 10))
    
    for i in range(0,axis[0]):
        for j in range (0,axis[1]):
            # Choose a random image
            index_img = np.random.randint(100)
            filename = images_folder_path + train + str(index_img) + '.jpg'
            # Read Image:
            img = Image.open(filename).convert('RGB')
            # To numpy
            img = np.asarray(img)
            # Plot
            axarr[i,j].imshow(img)
    plt.show()

In [ ]:
plot_images(axis=(3,3), images_folder_path=images_folder_train)

In [ ]:
plot_images(axis=(3,3), images_folder_path=images_folder_test, train=False)

## See Labels

In [ ]:
train_path_labels = train_path + 'train_v2.csv'
test_path_labels = test_path + 'test_v2_file_mapping.csv'

In [ ]:
# Train Images and labels
train_map = pd.read_csv(train_path_labels)
train_map

In [ ]:
pd.read_csv(test_path_labels)

# Create Dataset
* This is a multi labels task, so we need binary encode the tags
* We don't have labels for testing images so we need to split data of train in train and test
* We have many information so we need a data loader to wrap-up the data and avoid overload the GPU or RAM

## Binary Encode

In [ ]:
tags_encode = train_map.tags.str.get_dummies(sep=' ').columns
tags_encode

In [ ]:
# Select Tags
list_tags = ['agriculture', 'bare_ground', 'cultivation', 'habitation', 'primary', 'road', 'water']

## Filter images with tags 
* We have many tags, but we just want images with tags in list_tags so we filter

In [ ]:
def filter_item(tag_string):
    res = any(tag in tag_string for tag in list_tags)
    return res    

In [ ]:
train_map = train_map[train_map.tags.apply(lambda tag_string: filter_item(tag_string))]
train_map

## Binary Encode and desired columns 

In [ ]:
# Binary Encode
encode = train_map.tags.str.get_dummies(sep=' ')
train_map = pd.concat([train_map, encode], axis=1)
train_map.drop(columns=["tags"], inplace=True)
list_tags.insert(0,'image_name')
train_map = train_map[list_tags]
list_tags.pop(0)
train_map

### Add ".jpg" to image name

In [ ]:
train_map.image_name = train_map.image_name.apply(lambda name: name + '.jpg')
train_map

In [ ]:
for item in list_tags:
    print(f'the images with {item} are {len(train_map[train_map[item] == 1])}')

train_map[list_tags].sum().plot.bar()

# Train, Test, Validation Split

In [ ]:
from sklearn.model_selection import train_test_split

# Split Train and test
train, test = train_test_split(train_map, test_size=0.2, random_state=1)

# Split Train and validation
train, validation = train_test_split(train_map, test_size=0.1, random_state=1)

In [ ]:
train

In [ ]:
test

In [ ]:
validation

# Creating data Generator and Data Agumentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# We can't load all data in memory at once, so we use a DataGenerator
#Create instance of ImageDataGenerator Class
image_gen_train = ImageDataGenerator(
                    # Rescale
                    rescale=1./255,
                    # Rotate 30
                    rotation_range=30,
                    # Shift pixel values
                    width_shift_range=.15,
                    height_shift_range=.15,
                    # Flip all image
                    horizontal_flip=True,
                    # Random zoom
                    zoom_range=0.4
                    )
image_gen_test = ImageDataGenerator(rescale=1./255)
image_gen_valid = ImageDataGenerator(rescale=1./255)

In [ ]:
width = 256 # width = height
batch_size = 32

# Custom datagenerator
train_datagen = image_gen_train.flow_from_dataframe(dataframe=train,
                                                    directory=images_folder_train,
                                                    x_col='image_name',
                                                    y_col=list_tags,
                                                    batch_size=batch_size, #16,32,64...
                                                    seed=1,
                                                    shuffle=True,
                                                    class_mode="raw",
                                                    target_size=(width,width))
                                                                
test_datagen = image_gen_test.flow_from_dataframe(dataframe=test,
                                                    directory=images_folder_train,
                                                    x_col='image_name',
                                                    y_col=list_tags,
                                                    batch_size=batch_size, #16,32,64...
                                                    seed=1,
                                                    shuffle=False,
                                                    class_mode="raw",
                                                    target_size=(width,width))

valid_datagen = image_gen_valid.flow_from_dataframe(dataframe=validation,
                                                    directory=images_folder_train,
                                                    x_col='image_name',
                                                    y_col=list_tags,
                                                    batch_size=batch_size, #16,32,64...
                                                    seed=1,
                                                    shuffle=True,
                                                    class_mode="raw",
                                                    target_size=(width,width))



In [ ]:
import matplotlib.pyplot as plt
def plot_images_datagen(axis=(2,2), images=None):

    # Grid
    f, axarr = plt.subplots(axis[0], axis[1], figsize=(30/axis[1], 10))
    index = 0
    for i in range(0,axis[0]):
        for j in range (0,axis[1]):
            # Plot
            axarr[i,j].imshow(images[index])
            index += 1
    plt.show()

In [ ]:
# See Example of image datagenerator
example = image_gen_train.flow_from_dataframe(dataframe=validation,
                                                    directory=images_folder_train,
                                                    x_col='image_name',
                                                    y_col=list_tags,
                                                    batch_size=batch_size, #16,32,64...
                                                    seed=1,
                                                    shuffle=True,
                                                    class_mode="raw",
                                                    target_size=(width,width))

images, _ = next(example)
example_images = images[:9]
plot_images_datagen(axis=(3,3), images=example_images)

# Create Model
* <a href="https://arxiv.org/abs/1512.03385">Resnet </a>

In [ ]:
# See if GPU is aviable
import tensorflow as tf

gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

# Resnet50 Model

In [ ]:
from tensorflow.keras import applications

# See model
applications.resnet50.ResNet50(weights= None).summary()

# Create model with Function

In [ ]:
# Op 1:
from tensorflow.keras import applications
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

outs = len(list_tags)
def ResnetModel(outs = outs, freeze = False, pretrained = False):
    
    if pretrained:
        model_weights = 'imagenet'
    else:
        model_weights = None

    model = tf.keras.Sequential()
    model.add(applications.resnet50.ResNet50(weights= None, include_top=False, input_shape=(width,width,3)))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(outs, activation= 'sigmoid'))


    if freeze:
        # Training only top layers i.e. the layers which we have added in the end
        # Indicate whether the first layer should be trained/changed or not.
        model.layers[0].trainable = False  
    
    return model

In [ ]:
#model = ResnetModel(outs = outs, freeze = False, pretrained = False)
#model.summary()

# Create model with Class

In [ ]:
# Op 2:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


class MyModel(tf.keras.Model):

    def __init__(self, n_outputs=outs, pretrained=False, freeze=False, size = width, depth = 3):
        
        super(MyModel, self).__init__()
        
           
        if pretrained:
            self.model_weights = 'imagenet'
        else:
            self.model_weights = None
        
        # Download the architecture of ResNet50 with ImageNet weights
        self.resnet = applications.resnet50.ResNet50(include_top=False, weights=self.model_weights, input_shape= (width,width, depth))
        
        # Taking the output of the last convolution block in ResNet50
        self.res_out = self.resnet.output
        self.res_in = self.resnet.input
        
        self.GlobPoll = GlobalAveragePooling2D()
        
        # Adding a fully connected layer having 1024 neurons
        #self.fc1 = Dense(1024, activation='relu')
        
        # Sigmoid Out
        self.out = Dense(outs, activation='sigmoid')
        
        if freeze:
            # Training only top layers i.e. the layers which we have added in the end
            self.resnet.trainable = False

    def call(self, inputs):

        x = self.resnet(inputs)
        x = self.GlobPoll(x)
        #x = self.fc1(x)
        x = self.out(x)
        
        return x


# Instance of Model with default values (No pretrain, No Freeze)

In [ ]:
np.random.seed(1)
tf.random.set_seed(1234)

model = MyModel()
#model.build(input_shape=(None,256, 256, 3))
#model.summary()
#model.layers[0].trainable


In [ ]:
#np.random.seed(1)
#tf.random.set_seed(1234)

#model = ResnetModel()
#model.summary()

In [ ]:
#model = MyModel()
#model.load_weights(PATH + 'Models/ModelResnet50_Epoch10/Resnet50_tf_batch32_NoPretrained_epoch10')

# Calculate wights for unbalanced data

In [ ]:
positive_weights = {}
negative_weights = {}
for c in list_tags:
    positive_weights[c] = train.shape[0]/(2*np.count_nonzero(train[c]==1))
    negative_weights[c] = train.shape[0]/(2*np.count_nonzero(train[c]==0))
print(positive_weights)
print('----------------------')
print(negative_weights)



# Custom loss for unbalanced data

In [ ]:
# custon Binary Crossentropy
import tensorflow.keras.backend as K

def loss_fn(y_true,y_pred):
    
    y_true = tf.cast(y_true, tf.float32)
    
    #print(y_true.dtype)
    #print(y_pred.dtype)
    loss = 0
    loss -= (positive_weights['agriculture']*y_true[0]*K.log(y_pred[0]) + negative_weights['agriculture']*(1-y_true[0])*K.log(1-y_pred[0]))
    loss -= (positive_weights['bare_ground']*y_true[1]*K.log(y_pred[1]) + negative_weights['bare_ground']*(1-y_true[1])*K.log(1-y_pred[1]))
    loss -= (positive_weights['cultivation']*y_true[2]*K.log(y_pred[2]) + negative_weights['cultivation']*(1-y_true[2])*K.log(1-y_pred[2]))
    loss -= (positive_weights['habitation']*y_true[3]*K.log(y_pred[3]) + negative_weights['habitation']*(1-y_true[3])*K.log(1-y_pred[3]))
    loss -= (positive_weights['primary']*y_true[4]*K.log(y_pred[4]) + negative_weights['primary']*(1-y_true[4])*K.log(1-y_pred[4]))
    loss -= (positive_weights['road']*y_true[5]*K.log(y_pred[5]) + negative_weights['road']*(1-y_true[5])*K.log(1-y_pred[5]))
    loss -= (positive_weights['water']*y_true[6]*K.log(y_pred[6]) + negative_weights['water']*(1-y_true[6])*K.log(1-y_pred[6]))
    #print(loss)
    return loss

# Compile the Model

In [ ]:
#model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['categorical_accuracy','accuracy'])
model.compile(optimizer = tf.keras.optimizers.Adam(0.000003), loss = loss_fn, metrics = ['categorical_accuracy','accuracy'])

# Fit the model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=8, 
        verbose=1, mode='auto', restore_best_weights=True)

STEP_SIZE_TRAIN = train_datagen.n//train_datagen.batch_size
STEP_SIZE_VALID = valid_datagen.n//valid_datagen.batch_size
STEP_SIZE_TEST = test_datagen.n//test_datagen.batch_size



# https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/Model#fit
model.fit(x = train_datagen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_datagen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50,
                    callbacks=[monitor]
)


# Test the model

In [84]:
# Predict 
#test_datagen.reset()
pred=model.predict_generator(test_datagen,
                            steps=STEP_SIZE_TEST,
                            verbose=1)


/Users/davidrestrepo/opt/anaconda3/envs/SeguridadAlimentaria_GPU/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
2021-08-17 19:21:09.486652: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


239/239 [==============================] - 143s 592ms/step


In [85]:
print('the predictions are: ')
pred

the predictions are: 


array([[0.20797792, 0.00884381, 0.00669419, ..., 0.99647415, 0.05373644,
        0.2459486 ],
       [0.88146424, 0.04914488, 0.42343295, ..., 0.9932893 , 0.90127724,
        0.3021252 ],
       [0.6144184 , 0.03072709, 0.2935545 , ..., 0.99463344, 0.47456154,
        0.44160753],
       ...,
       [0.05268625, 0.00322772, 0.06081653, ..., 0.99970114, 0.0211084 ,
        0.06431799],
       [0.07344229, 0.00563491, 0.06689786, ..., 0.9996643 , 0.0302663 ,
        0.05437705],
       [0.08712693, 0.02146338, 0.09976102, ..., 0.97777236, 0.0429875 ,
        0.23085521]], dtype=float32)

In [86]:
print('the predictions are: ')
# Transform predictions to 0 or 1
round_pred = np.rint(pred)
round_pred

the predictions are: 


array([[0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 1., 1., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [87]:
print('the actual values are: ')
y_true = test_datagen.labels
y_true[:round_pred.shape[0],:]

the actual values are: 


array([[1, 0, 0, ..., 1, 1, 0],
       [1, 0, 0, ..., 1, 1, 1],
       [1, 0, 1, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [88]:
#tags
list_tags

['agriculture',
 'bare_ground',
 'cultivation',
 'habitation',
 'primary',
 'road',
 'water']

In [89]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def get_metrics(y_true=y_true, round_pred=round_pred, column=0):

    print(f'The column is {list_tags[column]}')
    y_true = y_true[:round_pred.shape[0],column]
    round_pred = round_pred[:,column]
    
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_true, round_pred)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_true, round_pred)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_true, round_pred)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_true, round_pred)
    print('F1 score: %f' % f1)

In [90]:
for i in range(len(list_tags)):
    print('---------')
    get_metrics(y_true, round_pred, i)
    print('---------')


---------
The column is agriculture
Accuracy: 0.798379
Precision: 0.699911
Recall: 0.644262
F1 score: 0.670935
---------
---------
The column is bare_ground
Accuracy: 0.979341
Precision: 0.416667
Recall: 0.032051
F1 score: 0.059524
---------
---------
The column is cultivation
Accuracy: 0.885722
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
---------
---------
The column is habitation
Accuracy: 0.913964
Precision: 0.634454
Recall: 0.209141
F1 score: 0.314583
---------
---------
The column is primary
Accuracy: 0.976857
Precision: 0.979619
Recall: 0.997056
F1 score: 0.988260
---------
---------
The column is road
Accuracy: 0.831851
Precision: 0.611225
Recall: 0.554314
F1 score: 0.581380
---------
---------
The column is water
Accuracy: 0.822306
Precision: 0.608156
Recall: 0.231600
F1 score: 0.335452
---------


/Users/davidrestrepo/opt/anaconda3/envs/SeguridadAlimentaria_GPU/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Save the model

In [91]:
# Save the weights (Class)
model.save_weights(PATH + 'Models/ModelResnet50Balanced/Resnet50_tf_batch32_NoPretrained_epoch50')
#model.save_weights(PATH + 'Models/ModelResnet50Balanced/Resnet50_tf_batch32_NoPretrained_epoch10')

In [75]:
# Save Model (function)
#model.save(PATH + 'Models/Resnet50_tf_batch32_NoPretrained.h5')

# Load the model

In [67]:
# Load moMyModelModelModell class
model2 = MyModel()
model2.load_weights(PATH+'Models/ModelResnet50Balanced/Resnet50_tf_batch32_NoPretrained')
#model2.load_weights(PATH+'Models/ModelResnet50_Epoch10/Resnet50_tf_batch32_NoPretrained_epoch10')

#from tensorflow.keras.models import load_model
# Load Model (function) 
#new_model = load_model(PATH + 'Models/ModelVGG16/VGG16_tf_batch32_NoPretrained.h5')
#new_model.summary()

# Predictions

In [68]:
random = np.random.uniform(low=0.0, high=1.0, size=(1,256,256,3))
model.predict(random)

array([[0.10148496, 0.0047991 , 0.00148913, 0.9999881 , 0.5797755 ,
        0.999156  , 0.02957684]], dtype=float32)

In [ ]:
# Guardar el Modelo
#model.save(PATH + 'Models/Resnet50_tf_batch32_NoPretrained.h5')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_6 ( (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 14343     
Total params: 23,602,055
Trainable params: 23,548,935
Non-trainable params: 53,120
_________________________________________________________________
